I initialize the list with the function below, based on the algorithm on slide 9 of the pdf. 

In [ ]:
function initialize_list(X::particle)
    dx = zeros(3)
    mc = zeros(Int, 3)
    head = zeros(Int, Ncel)
    list = zeros(Int, nPart)
    
    for i in 1:nPart
        for j in 1:dim
            dx[j] = X.q[j,i] + L/2
            
            if dx[j] < 0
                dx[j] += L
            elseif dx[j] > L
                dx[j] -= L
            end
            
            mc[j] = floor(Int, dx[j]/L*M)
            
            if mc[j] < 0
                mc[j] = 0
            elseif mc[j] >= M
                mc[j] = M-1
            end
        end
        c = 1 + mc[1] + mc[2]*M + mc[3]*M^2

        list[i] = head[c]
        head[c] = i
    end
    
    return head, list
end

The function below decodes the list and computes the force, based on the algorithm from page 10 of the pdf. 

In [ ]:
function compute_force(X::particle)
    X.f = zeros(dim, nPart)
    clear_list()
    head, list = initialize_list(X)
    mc = zeros(Int, 3)
    dx = zeros(3)
    ci = zeros(Int, 3)

    for i in 1:nPart # Loop over particles
        for j in 1:dim 
            dx[j] = X.q[j,i] + L/2
            mc[j] = floor(Int, dx[j]/L*M)
        end
        
        for ci[1] in (mc[1]-1):(mc[1]+1), # Loop over 27 neighboring cells
            ci[2] in (mc[2]-1):(mc[2]+1),
            ci[3] in (mc[3]-1):(mc[3]+1)

            for j in 1:dim
                if ci[j] < 0
                    ci[j] = 0
                elseif ci[j] >= M
                    ci[j] = M-1
                end
            end
            c = 1 + ci[1] + ci[2]*M + ci[3]*M^2
            j = head[c]

            while j != 0
                if i < j # Don't count double pairs
                    dr = X.q[:,i] - X.q[:,j]
                    r = distPBC(dr)
                    ff = LJ_force(r) 
                    X.f[:,i] += ff*r
                    X.f[:,j] -= ff*r
                    
                end
                j = list[j]
            end
        end
    end
   
    return X.f
end